In [1]:
#Nikolaos Giannopoulos AM 5199
#Georgios Strouggis AM 5357

#All the imports happen here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import torch as t
import torch.nn as nn
import torch.optim as optim


#Load the data
train_data = pd.read_csv('fashion-mnist_train.csv')
test_data = pd.read_csv('fashion-mnist_test.csv')

#Make sure it got loaded correctly
print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")

#Print a few data form each data set
print("\nFirst few rows of training data:")
print(train_data.head())
print("\nFirst few rows of testing data:")
print(test_data.head())


#Split images and labels as X and Y
X_train = train_data.drop(columns=['label'], axis=1).values
X_test = test_data.drop(columns=['label'], axis=1).values
Y_train = train_data['label'].values
Y_test = test_data['label'].values

#Use Standard scaler for the images
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Finallize Y
y_train = Y_train
y_test = Y_test

#Pictures of 28x28
x_train = X_train_scaled.reshape((-1, 28, 28))
x_test = X_test_scaled.reshape((-1, 28, 28))

#View Train Data as pictures
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(x_train[i], cmap='viridis')
plt.show()

In [ ]:
#Convert the data from numpy to tensors
x_train_tensor = t.tensor(x_train, dtype=t.float32)
x_test_tensor = t.tensor(x_test, dtype=t.float32)

#Reshape the tensors to match the expected input shape 
x_train_tensor = x_train_tensor.unsqueeze(1)
x_test_tensor = x_test_tensor.unsqueeze(1)

#Apply max polling to train/test data
x_train_polled = t.max_pool2d(x_train_tensor, kernel_size=4)
x_test_polled = t.max_pool2d(x_test_tensor, kernel_size=4)

#Convert the pooled tensors back to numpy arrays
x_train_np = x_train_polled.squeeze(1).numpy()  
x_test_np = x_test_polled.squeeze(1).numpy()

#Print to test if all went accordingly
print(f"Train data shape: {x_train_np.shape}")
print(f"Test data shape: {x_test_np.shape}")

#Array of polled train/test data
x_train_array = x_train_np.reshape(x_train_np.shape[0], -1)
x_test_array = x_test_np.reshape(x_test_np.shape[0], -1)

#Print to test that the conversation to array worked
print(f"Train data array shape: {x_train_array.shape}")
print(f"Test data array shape: {x_test_array.shape}")


#Print of the polled images to certify the max polling
for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(x_train_np[i], cmap='viridis')
plt.show()


In [ ]:
#Define different values for K
k_values = [1, 3, 5]

#Loop for different K values, then create classifier, train it and finally test it
for k in k_values:
    knn_classifier = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    knn_classifier.fit(x_train_array, y_train)
    accuracy = knn_classifier.score(x_test_array, y_test)
    
#Prints of report for each value
    print(f"Accuracy with K={k}: {accuracy}\n")
    print(f"Classification Report: {classification_report(y_test, y_pred=knn_classifier.predict(x_test_array))}")
    print("----------------------------")

In [ ]:
#Create and train the Decision Tree with max depth of 10
decision_tree = DecisionTreeClassifier(max_depth=10)
decision_tree.fit(x_train_array, y_train)

#Create and train the Random Forest with the 100 estimators
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(x_train_array, y_train)
#Forest accuracy
accuracy = random_forest.score(x_test_array, y_test)
print(f"Accuracy with Random Forest: {accuracy}\n")

#Plot creator for the Decision Tree
print("The Decision Tree:\n")
plt.figure(figsize=(20, 10))
plot_tree(decision_tree, filled=True, feature_names=[str(i) for i in range(49)], class_names=[str(i) for i in range(10)])
plt.show()



In [ ]:

#Definiton of C and parameter values
C_values = [1, 10, 100]
param_values = [0.02, 0.1, 1]

#Due to the complexity we are forced to use a subset of data
subset = []
for label in np.unique(y_train):
    label_values = np.where(y_train == label)[0]
    np.random.shuffle(label_values)
    num_train = int(len(label_values) * 1)
    subset.extend(label_values[:num_train])
x_train_subset = x_train_array[subset]
y_train_subset = y_train[subset]

#To improve data convergence we will use a scaler to preprocess it
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train_subset)
x_test_scaled = scaler.transform(x_test_array)


print("Number of samples in subsampled training set:", x_train_subset.shape)

#Start of loop for C values for Linear and RBF kernel
for c in C_values:
    linear_svm = SVC(kernel='linear', C=c, max_iter=500) #Classifier for linear SVM
    linear_svm.fit(x_train_scaled, y_train_subset) #Train linear SVM
    
    #See the accuracy of linear SVM
    accuracy = linear_svm.score(x_test_scaled, y_test)
    print(f"Accuracy with Linear SVM with C= {c}: {accuracy}\n")
    
    best_p = 0
    best_accuracy = 0
    #Start of loop for Parameter values for the RBF kernel
    for p in param_values:
        rbf_kernel_svm = SVC(kernel='rbf', gamma=p, C=c, max_iter=500)
        rbf_kernel_svm.fit(x_train_scaled, y_train_subset)
        rbf_accuracy = rbf_kernel_svm.score(x_test_scaled, y_test)
        if rbf_accuracy>best_accuracy:
            best_accuracy = rbf_accuracy
            best_p = p
        print(f"Accuracy with RBF SVM with C= {c} and P={p}: {rbf_accuracy}\n")
        print("---------------------------------------------")
    print(f"The best accuracy belongs to P={best_p}: {best_accuracy}")
    print("---------------------------------------------")
    

In [ ]:
#CROSS ENTROPY VERSION WHICH WORKS!
class FeedForwardNeuralNetwork(nn.Module):
    def __init__(self, input_size, num_classes):
        super(FeedForwardNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 100) #First layer with 100
        self.fc2 = nn.Linear(100, 100) #Second layer
        self.fc3 = nn.Linear(100, 50) #Third layer
        self.fc4 = nn.Linear(50, num_classes) #Output layer
        self.relu = nn.LeakyReLU() #LeakyRelU function
        self.softmax = nn.Softmax(dim=1) #Softmax function
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.relu(self.fc2(out))
        out = self.relu(self.fc3(out))
        out = self.fc4(out)
        return self.softmax(out)
    
#Transform to Tensors
x_train_tensor = t.tensor(x_train_array, dtype=t.float32)
x_test_tensor = t.tensor(x_test_array, dtype=t.float32)
y_train_tensor = t.tensor(y_train, dtype=t.long)
y_test_tensor = t.tensor(y_test, dtype=t.long)

#Define the input_size and num_classes and then create the FeedForwardNeuralNetwork
input_size = x_train_tensor.shape[1]
num_classes = len(t.unique(y_train_tensor))
model = FeedForwardNeuralNetwork(input_size, num_classes)

#Parameters print
num_param = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {num_param}\n")

#Loss and optimizer function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Batch and epochs initilizer with the loss and accuracy arrays
epochs = 100
batch_size = 50
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []



#Define the scallers for features and target variable
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
#Perform grid search
best_score = float('inf')
best_params = {}



#Start of training loop
for epoch in range(epochs):
    epoch_train_loss = 0.0
    epoch_train_correct = 0.0
    epoch_test_loss = 0.0
    epoch_test_correct = 0.0
    
    model.train()
    for i in range(0, len(x_train_tensor), batch_size):
        batch_x = t.tensor(x_train_tensor[i:i+batch_size], dtype=t.float32)
        batch_y = t.tensor(y_train_tensor[i:i+batch_size], dtype=t.long)
        optimizer.zero_grad()
        outputs = model(batch_x) #Trainer definition
        loss = criterion(outputs, batch_y) #Loss definition
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item() #Loss counter
        
        _, predicted = t.max(outputs, 1) #Accuracy counter
        epoch_train_correct += (predicted == batch_y).sum().item()

        
        
    #Model evaluation
    model.eval()
    with t.no_grad():
        for j in range(0, len(x_test_tensor), batch_size):
            batch_test_x = t.tensor(x_test_tensor[j:j+batch_size], dtype=t.float32)
            batch_test_y = t.tensor(y_test_tensor[j:j+batch_size], dtype=t.long)
            y_pred = model(batch_test_x) #Trainer for test
            loss_test = criterion(y_pred, batch_test_y) #Loss for test
            epoch_test_loss += loss_test.item() #Loss counter
            _, predicted_test = t.max(y_pred, 1) #Accuracy counter
            epoch_test_correct += (predicted_test == batch_test_y).sum().item()
        
        
    #Data classification to arrays for plot visualization
    epoch_train_loss /= len(x_train_tensor)
    epoch_train_accuracy = epoch_train_correct / len(x_train_tensor)
    epoch_test_loss /= len(x_test_tensor)
    epoch_test_accuracy = epoch_test_correct / len(x_test_tensor)

    train_losses.append(epoch_train_loss)
    train_accuracies.append(epoch_train_accuracy)
    test_losses.append(epoch_test_loss)
    test_accuracies.append(epoch_test_accuracy)

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_accuracy:.4f}, Test Loss: {epoch_test_loss:.4f}, Test Acc: {epoch_test_accuracy:.4f}")

# Plot loss and accuracy
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
   



In [ ]:
#MSELOSS VERSION WHICH DOESNT WORK
class FeedForwardNeuralNetworker(nn.Module):
    def __init__(self, input_size, num_classes):
        super(FeedForwardNeuralNetworker, self).__init__()
        self.fc1 = nn.Linear(input_size, 100) #First layer with 100
        self.fc2 = nn.Linear(100, 100) #Second layer
        self.fc3 = nn.Linear(100, 50) #Third layer
        self.fc4 = nn.Linear(50, num_classes) #Output layer
        self.relu = nn.LeakyReLU() #LeakyRelU function
        self.softmax = nn.Softmax(dim=1) #Softmax function
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.relu(self.fc2(out))
        out = self.relu(self.fc3(out))
        out = self.fc4(out)
        return self.softmax(out)
    


#Define the input_size and num_classes and then create the FeedForwardNeuralNetworker
input_size = x_train_tensor.shape[1]
num_classes = len(t.unique(y_train_tensor))
model = FeedForwardNeuralNetworker(input_size, num_classes)

num_param = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {num_param}\n")

#Loss and optimizer function
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#Batch and epochs initilizer with the loss and accuracy arrays
epochs = 100
batch_size = 50
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []



#Define the scallers for features and target variable
scaler_x = MinMaxScaler()
x_train_scaled = scaler_x.fit_transform(x_train_array)
x_test_scaled = scaler_x.transform(x_test_array)

# Scale output values if needed
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()

# Transform to Tensors
x_train_tensor = t.tensor(x_train_scaled, dtype=t.float32)
x_test_tensor = t.tensor(x_test_scaled, dtype=t.float32)
y_train_tensor = t.tensor(y_train_scaled, dtype=t.float32)
y_test_tensor = t.tensor(y_test_scaled, dtype=t.float32)

#Perform grid search
best_score = float('inf')
best_params = {}



#Start of training loop
#num_batches = len(x_train_tensor) // batch_size
for epoch in range(epochs):
    epoch_train_loss = 0.0
    epoch_train_correct = 0.0
    epoch_test_loss = 0.0
    epoch_test_correct = 0.0
    
    model.train()
    for i in range(0, len(x_train_tensor), batch_size):
        batch_x = t.tensor(x_train_tensor[i:i+batch_size], dtype=t.float32)
        batch_y = t.tensor(y_train_tensor[i:i+batch_size], dtype=t.float32).view(-1, 1)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
        _, predicted = t.max(outputs, 1)
        epoch_train_correct += (predicted == batch_y).sum().item()
    #Model evaluation
    model.eval()
    with t.no_grad():
        for j in range(0, len(x_test_tensor), batch_size):
            batch_test_x = t.tensor(x_test_tensor[j:j+batch_size], dtype=t.float32)
            batch_test_y = t.tensor(y_test_tensor[j:j+batch_size], dtype=t.float32).view(-1, 1)
            y_pred = model(batch_test_x)
            loss_test = criterion(y_pred, batch_test_y)
            epoch_test_loss += loss_test.item()
            _, predicted_test = t.max(y_pred, 1)
            epoch_test_correct += (predicted_test == batch_test_y).sum().item()
        
    
    epoch_train_loss /= len(x_train_tensor)
    epoch_train_accuracy = epoch_train_correct / len(x_train_tensor)
    epoch_test_loss /= len(x_test_tensor)
    epoch_test_accuracy = epoch_test_correct / len(x_test_tensor)

    train_losses.append(epoch_train_loss)
    train_accuracies.append(epoch_train_accuracy)
    test_losses.append(epoch_test_loss)
    test_accuracies.append(epoch_test_accuracy)

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_accuracy:.4f}, Test Loss: {epoch_test_loss:.4f}, Test Acc: {epoch_test_accuracy:.4f}")

# Plot loss and accuracy
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
   
        
        
        

In [ ]:

class ConvolutionalNeuralNetworkTest(nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNetworkTest, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1) #Input to first layer
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1) #First to second layer
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1) #Second to third
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #Max Pooling layer
        self.flatten = nn.Flatten() #Important for fc1
        self.fc1 = nn.Linear(64 * 14 * 14, 100)  # Adjusted input size to match the output of conv2
        self.dropout = nn.Dropout(0.3) #Dropout layer
        self.fc2 = nn.Linear(100, 10)  # Output size matches the number of classes
        self.softmax = nn.Softmax() #Softmax layer
    
    def forward(self, x):
        out = t.relu(self.conv1(x))
        out = t.relu(self.conv2(out))
        out = self.pool(out)
        out = t.relu(self.conv3(out))
        out = self.flatten(out)
        out = t.relu(self.fc1(out))
        out = self.dropout(out)
        out = self.fc2(out)
        return self.softmax(out)
        
#Regulate the shape of the images before the transformation to tensor
x_train_imager = x_train.reshape((-1, 28, 28, 1))
x_test_imager = x_test.reshape((-1, 28, 28, 1))

#Turn to tensors the train and test data with the correct order of data
x_train_torch = t.tensor(x_train_imager, dtype=t.float32).permute(0, 3, 1, 2)
x_test_torch = t.tensor(x_test_imager, dtype=t.float32).permute(0, 3, 1, 2)
y_train_torch = t.tensor(y_train, dtype=t.long)
y_test_torch = t.tensor(y_test, dtype=t.long)

#Model, criterion and optimizer definition
modeler = ConvolutionalNeuralNetworkTest()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modeler.parameters(), lr=0.001)

#Important definitions
epochs = 100
batch_size = 50
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

#Start of training loop
for epoch in range(epochs):
    epoch_train_loss = 0.0
    epoch_train_correct = 0.0
    epoch_test_loss = 0.0
    epoch_test_correct = 0.0
    modeler.train()
    for i in range(0, len(x_train_torch), batch_size):
        optimizer.zero_grad()
        batch_x = x_train_torch[i:i+batch_size]
        batch_y = y_train_torch[i:i+batch_size]
        
        outputs = modeler(batch_x) #Train definition
        loss = criterion(outputs, batch_y) #Loss definition
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item() #Loss counter
        _, predicted = t.max(outputs, 1) #Accuracy counter
        epoch_train_correct += (predicted == batch_y).sum().item()
        
    #Start of test loop
    modeler.eval()
    with t.no_grad():
        for j in range(0, len(x_test_torch), batch_size):
            batch_x = x_test_torch[j:j+batch_size]
            batch_y = y_test_torch[j:j+batch_size]
            outputs = modeler(batch_x) #Trainer for test
            loss = criterion(outputs, batch_y) #Loss for test
            epoch_test_loss += loss.item() #Loss counter
            _, predicted = t.max(outputs, 1) #Accuracy counter
            epoch_test_correct += (predicted == batch_y).sum().item()
    
    #Data classification to arrays for plot visualization
    epoch_train_loss /= len(x_train_torch)
    epoch_train_accuracy = epoch_train_correct / len(x_train_torch)
    epoch_test_loss /= len(x_test_torch)
    epoch_test_accuracy = epoch_test_correct / len(x_test_torch)
    
    train_losses.append(epoch_train_loss)
    train_accuracies.append(epoch_train_accuracy)
    test_losses.append(epoch_test_loss)
    test_accuracies.append(epoch_test_accuracy)
    
    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_accuracy:.4f}, Test Loss: {epoch_test_loss:.4f}, Test Acc: {epoch_test_accuracy:.4f}")
    

# Plot loss and accuracy
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()